In [1]:
import pyodbc

In [ ]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=testdb;UID=me;PWD=pass')
cursor = cnxn.cursor()
cursor.execute(
f"""
SELECT
s.TREEELEMID,
s.PARENTID,
s.CONTAINERTYPE,
s.NAME,
s.ELEMENTENABLE, --Disabled/Enabled
s.PARENTENABLE, --Sometimes we have all asset disabled. This will be visible here
s.ChannelEnable,
s.HIERARCHYTYPE,
s.TBLSETID,
s.BRANCHLEVEL,
put.VALUESTRING as PointUnitType, -- Units. Should be according to names of MP
fr.VALUESTRING as FilterEnvelope, -- Filter range for enveloped acceleration.
psut.VALUESTRING as PointSensorUnitType, --Which sensor used. Don't know if it's necessary
po.VALUESTRING as PointOrientation, --Orientation of the point (H,V,A,R)
pl.VALUESTRING as PointLocation, --Location of the point
dad.VALUESTRING as DADType, -- DAD type
pcct.VALUESTR as FilterKey, --Filter Key 
sa.SCALARALRMID,
sa.ALARMMETHOD,
sa.DANGERHI,
sa.DANGERLO,
sa.ALERTHI,
sa.ALERTLO,
sa.ENABLEALERTHI,
sa.ENABLEALERTLO,
sa.ENABLEDANGERHI,
sa.ENABLEDANGERLO,
g.PRIORITY as NodePriority
FROM
skfuser1.TREEELEM s
-- add overall alarms information 
LEFT JOIN skfuser1.SCALARALARM sa ON sa.SCALARALRMID = (SELECT ALARMID FROM skfuser1.ALARMASSIGN aa WHERE aa.ELEMENTID = s.TREEELEMID and aa.TYPE = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASAT_Overall') and aa.CHANNEL = 1)
-- add additional point settings fields
LEFT JOIN skfuser1.POINT put ON s.CONTAINERTYPE = 4 AND put.ELEMENTID = s.TREEELEMID and put.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Full_Scale_Unit') --point unit type. (Need full list of units that are allowed here!)
LEFT JOIN skfuser1.POINT psut ON s.CONTAINERTYPE = 4 AND psut.ELEMENTID = s.TREEELEMID and psut.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Sensor') --point sensor type
LEFT JOIN skfuser1.POINT po ON s.CONTAINERTYPE = 4 AND po.ELEMENTID = s.TREEELEMID and po.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Orientation') --point orientation
LEFT JOIN skfuser1.POINT pl ON s.CONTAINERTYPE = 4 AND pl.ELEMENTID = s.TREEELEMID and pl.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Location') --point location (01-99)
LEFT JOIN skfuser1.POINT dad ON s.CONTAINERTYPE = 4 AND dad.ELEMENTID = s.TREEELEMID and dad.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Dad_Id') --dad type (Need to do reverse engineering!)
LEFT JOIN skfuser1.POINT fr ON s.CONTAINERTYPE = 4 AND fr.ELEMENTID = s.TREEELEMID and fr.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Input_Filter_Range') --filter range (Value - 20599 = Filter range)
LEFT JOIN skfuser1.GROUPTBL g ON s.TREEELEMID = g.ELEMENTID 
-- add filter keys details
LEFT JOIN (SELECT DISTINCT pt.ELEMENTID, ct.VALUESTR FROM skfuser1.POINTCAT pt, skfuser1.CATEGORY ct WHERE pt.CATEGORYID=ct.CATEGORYID AND ct.VALUESTR LIKE '*%') as pcct ON s.TREEELEMID = pcct.ELEMENTID  --Filter Key
WHERE
s.TBLSETID = {x}
AND s.HIERARCHYTYPE = 1 -- only standard hierarchy node, no route nodes
AND s.PARENTID != 2147000000 -- deleted/invalid nodes have this id
"""
)
rows = cursor.fetchall()
cnxn.close()

In [3]:
for x in [277, 54, 26]:
    print(
    f"""
    SELECT
        s.TREEELEMID,
        s.PARENTID,
        s.CONTAINERTYPE,
        s.NAME,
        s.ELEMENTENABLE, --Disabled/Enabled
        s.PARENTENABLE, --Sometimes we have all asset disabled. This will be visible here
        s.ChannelEnable,
        s.HIERARCHYTYPE,
        s.TBLSETID,
        s.BRANCHLEVEL,
        put.VALUESTRING as PointUnitType, -- Units. Should be according to names of MP
        fr.VALUESTRING as FilterEnvelope, -- Filter range for enveloped acceleration.
        psut.VALUESTRING as PointSensorUnitType, --Which sensor used. Don't know if it's necessary
        po.VALUESTRING as PointOrientation, --Orientation of the point (H,V,A,R)
        pl.VALUESTRING as PointLocation, --Location of the point
        dad.VALUESTRING as DADType, -- DAD type
        pcct.VALUESTR as FilterKey, --Filter Key 
        sa.SCALARALRMID,
        sa.ALARMMETHOD,
        sa.DANGERHI,
        sa.DANGERLO,
        sa.ALERTHI,
        sa.ALERTLO,
        sa.ENABLEALERTHI,
        sa.ENABLEALERTLO,
        sa.ENABLEDANGERHI,
        sa.ENABLEDANGERLO,
        g.PRIORITY as NodePriority
    FROM
        skfuser1.TREEELEM s
        -- add overall alarms information 
        LEFT JOIN skfuser1.SCALARALARM sa ON sa.SCALARALRMID = (SELECT ALARMID FROM skfuser1.ALARMASSIGN aa WHERE aa.ELEMENTID = s.TREEELEMID and aa.TYPE = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASAT_Overall') and aa.CHANNEL = 1)
        -- add additional point settings fields
        LEFT JOIN skfuser1.POINT put ON s.CONTAINERTYPE = 4 AND put.ELEMENTID = s.TREEELEMID and put.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Full_Scale_Unit') --point unit type. (Need full list of units that are allowed here!)
        LEFT JOIN skfuser1.POINT psut ON s.CONTAINERTYPE = 4 AND psut.ELEMENTID = s.TREEELEMID and psut.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Sensor') --point sensor type
        LEFT JOIN skfuser1.POINT po ON s.CONTAINERTYPE = 4 AND po.ELEMENTID = s.TREEELEMID and po.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Orientation') --point orientation
        LEFT JOIN skfuser1.POINT pl ON s.CONTAINERTYPE = 4 AND pl.ELEMENTID = s.TREEELEMID and pl.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Location') --point location (01-99)
        LEFT JOIN skfuser1.POINT dad ON s.CONTAINERTYPE = 4 AND dad.ELEMENTID = s.TREEELEMID and dad.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Dad_Id') --dad type (Need to do reverse engineering!)
        LEFT JOIN skfuser1.POINT fr ON s.CONTAINERTYPE = 4 AND fr.ELEMENTID = s.TREEELEMID and fr.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Input_Filter_Range') --filter range (Value - 20599 = Filter range)
        LEFT JOIN skfuser1.GROUPTBL g ON s.TREEELEMID = g.ELEMENTID 
        -- add filter keys details
        LEFT JOIN (SELECT DISTINCT pt.ELEMENTID, ct.VALUESTR FROM skfuser1.POINTCAT pt, skfuser1.CATEGORY ct WHERE pt.CATEGORYID=ct.CATEGORYID AND ct.VALUESTR LIKE '*%') as pcct ON s.TREEELEMID = pcct.ELEMENTID  --Filter Key
    WHERE
        s.TBLSETID = {x}
        AND s.HIERARCHYTYPE = 1 -- only standard hierarchy node, no route nodes
        AND s.PARENTID != 2147000000 -- deleted/invalid nodes have this id
    """
    )


    SELECT
        s.TREEELEMID,
        s.PARENTID,
        s.CONTAINERTYPE,
        s.NAME,
        s.ELEMENTENABLE, --Disabled/Enabled
        s.PARENTENABLE, --Sometimes we have all asset disabled. This will be visible here
        s.ChannelEnable,
        s.HIERARCHYTYPE,
        s.TBLSETID,
        s.BRANCHLEVEL,
        put.VALUESTRING as PointUnitType, -- Units. Should be according to names of MP
        fr.VALUESTRING as FilterEnvelope, -- Filter range for enveloped acceleration.
        psut.VALUESTRING as PointSensorUnitType, --Which sensor used. Don't know if it's necessary
        po.VALUESTRING as PointOrientation, --Orientation of the point (H,V,A,R)
        pl.VALUESTRING as PointLocation, --Location of the point
        dad.VALUESTRING as DADType, -- DAD type
        pcct.VALUESTR as FilterKey, --Filter Key 
        sa.SCALARALRMID,
        sa.ALARMMETHOD,
        sa.DANGERHI,
        sa.DANGERLO,
        sa.ALERTHI,
        sa.ALERTLO,
        sa.ENABLEALERTHI,
     

In [ ]:
def create_connection(server = '', db = '', uid = '', pwd = ''):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    credentials = f'DRIVER=\{ODBC Driver 17 for SQL Server\};SERVER={server};DATABASE={db};UID={uid};PWD={pwd}'
    conn = None
    try:
        conn = pyodbc.connect(credentials)
    except Error as e:
        print(e)

    return conn

def get_data(conn, tablset = 1):
    #Preparation of the quesry for data fetching
    cur = conn.cursor()
    # Main query for data fetching with imputed tablsetid
    query = f""" 
    SELECT
        s.TREEELEMID,
        s.PARENTID,
        s.CONTAINERTYPE,
        s.NAME,
        s.ELEMENTENABLE, --Disabled/Enabled
        s.PARENTENABLE, --Sometimes we have all asset disabled. This will be visible here
        s.ChannelEnable,
        s.HIERARCHYTYPE,
        s.TBLSETID,
        s.BRANCHLEVEL,
        put.VALUESTRING as PointUnitType, -- Units. Should be according to names of MP
        fr.VALUESTRING as FilterEnvelope, -- Filter range for enveloped acceleration.
        psut.VALUESTRING as PointSensorUnitType, --Which sensor used. Don't know if it's necessary
        po.VALUESTRING as PointOrientation, --Orientation of the point (H,V,A,R)
        pl.VALUESTRING as PointLocation, --Location of the point
        dad.VALUESTRING as DADType, -- DAD type
        pcct.VALUESTR as FilterKey, --Filter Key 
        sa.SCALARALRMID,
        sa.ALARMMETHOD,
        sa.DANGERHI,
        sa.DANGERLO,
        sa.ALERTHI,
        sa.ALERTLO,
        sa.ENABLEALERTHI,
        sa.ENABLEALERTLO,
        sa.ENABLEDANGERHI,
        sa.ENABLEDANGERLO,
        g.PRIORITY as NodePriority
    FROM
        skfuser1.TREEELEM s
        -- add overall alarms information 
        LEFT JOIN skfuser1.SCALARALARM sa ON sa.SCALARALRMID = (SELECT ALARMID FROM skfuser1.ALARMASSIGN aa WHERE aa.ELEMENTID = s.TREEELEMID and aa.TYPE = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASAT_Overall') and aa.CHANNEL = 1)
        -- add additional point settings fields
        LEFT JOIN skfuser1.POINT put ON s.CONTAINERTYPE = 4 AND put.ELEMENTID = s.TREEELEMID and put.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Full_Scale_Unit') --point unit type. (Need full list of units that are allowed here!)
        LEFT JOIN skfuser1.POINT psut ON s.CONTAINERTYPE = 4 AND psut.ELEMENTID = s.TREEELEMID and psut.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Sensor') --point sensor type
        LEFT JOIN skfuser1.POINT po ON s.CONTAINERTYPE = 4 AND po.ELEMENTID = s.TREEELEMID and po.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Orientation') --point orientation
        LEFT JOIN skfuser1.POINT pl ON s.CONTAINERTYPE = 4 AND pl.ELEMENTID = s.TREEELEMID and pl.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Location') --point location (01-99)
        LEFT JOIN skfuser1.POINT dad ON s.CONTAINERTYPE = 4 AND dad.ELEMENTID = s.TREEELEMID and dad.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Dad_Id') --dad type (Need to do reverse engineering!)
        LEFT JOIN skfuser1.POINT fr ON s.CONTAINERTYPE = 4 AND fr.ELEMENTID = s.TREEELEMID and fr.FIELDID = (SELECT REGISTRATIONID FROM skfuser1.REGISTRATION as r WHERE r.SIGNATURE  = 'SKFCM_ASPF_Input_Filter_Range') --filter range (Value - 20599 = Filter range)
        LEFT JOIN skfuser1.GROUPTBL g ON s.TREEELEMID = g.ELEMENTID 
        -- add filter keys details
        LEFT JOIN (SELECT DISTINCT pt.ELEMENTID, ct.VALUESTR FROM skfuser1.POINTCAT pt, skfuser1.CATEGORY ct WHERE pt.CATEGORYID=ct.CATEGORYID AND ct.VALUESTR LIKE '*%') as pcct ON s.TREEELEMID = pcct.ELEMENTID  --Filter Key
    WHERE
        s.TBLSETID = {tablset}
        AND s.HIERARCHYTYPE = 1 -- only standard hierarchy node, no route nodes
        AND s.PARENTID != 2147000000 -- deleted/invalid nodes have this id
    """
    #Executing the query
    data = pd.read_sql(query,conn)  # without parameters [non-prepared statement]
    conn.close()
    return data